# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [2]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? False


## Helpers (use or modify)

In [3]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [4]:
import yfinance as yf
import datetime as dt

def ts():
    return dt.datetime.now().strftime("%Y%m%d-%H%M")

ticker = "NVDA"
df_api = yf.download(ticker, period="3mo", interval="1d").reset_index()[["Date","Close"]]
df_api.columns = ["date","close"]

path_api = f"data/raw/api_yf_{ticker}_{ts()}.csv"
df_api.to_csv(path_api, index=False)
print("Saved:", path_api)

v_api = validate(df_api, ['date','close']); v_api

/var/folders/hj/0fkcndtd5h70rxxk2mnp3tx80000gn/T/ipykernel_34754/1340610700.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(ticker, period="3mo", interval="1d").reset_index()[["Date","Close"]]
[*********************100%***********************]  1 of 1 completed

Saved: data/raw/api_yf_NVDA_20250817-1454.csv


{'missing': [], 'shape': (63, 2), 'na_total': 0}

In [8]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha', symbol= 'NVDA')

Saved data/raw/api_source-alpha_symbol-NVDA_20250817-1454.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [9]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'# TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (880, 8), 'na_total': 758}

In [10]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250817-1454.csv


## Documentation
- API Source: Yahoo Finance via `yfinance`  
  - Example: `ticker=NVDA`, `period=3mo`, `interval=1d`  
  - Provides OHLCV data, used `date` + `close`

- Scrape Source: Wikipedia — [List of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)  
  - Parsed main table into DataFrame with 8 columns (Ticker, Security, Sector, etc.)

- Assumptions & risks:  
  - API may enforce **rate limits** or change schema (e.g., column naming)  
  - Scrape may break if **HTML selectors** or table layout change  
  - Some `NA` values may appear; acceptable for exercise but would need handling in production

- `.env`:  
  - Verified that `.env` is **not committed** to GitHub (`.gitignore` covers it)
